In [1]:
import pandas as pd

# Load the existing CSV file
log_file_path = 'identithem_logs_testfiles(3days)_unsupervisedmethods.csv'
logs_df = pd.read_csv(log_file_path)



In [2]:
logs_df

,ip,status,size,n_nans,is_bot,n_requests,n_bots,n_errors,average_size,average_timedif,...,is_attack,is_blacklist,anomaly_score,kmeans_result,DBSCAN_Anomaly,Isolation Forest Anomaly Prediction,LOF_Prediction,score,anomaly,is_Anomaly
0,5488240137,301,538,1,1,2,2,0,561.000000,1.000000,...,0,0,0.026291,0,1,0,-1,-50.081634,0,1
1,5488240137,301,584,1,1,2,2,0,561.000000,1.000000,...,0,0,0.042150,0,1,0,-1,-50.443050,0,1
2,11024920167,200,8106,1,1,20,20,0,19415.650000,0.105263,...,0,0,0.053346,0,1,0,-1,-57.340641,0,1
3,11024920167,200,1871,0,1,20,20,0,19415.650000,0.105263,...,0,0,0.013480,0,1,0,-1,-51.998813,0,1
4,11024920167,200,190599,0,1,20,20,0,19415.650000,0.105263,...,0,0,0.017192,0,1,0,-1,-91.228917,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337351,19515412693,200,454,0,1,38,38,0,12278.447368,0.027027,...,0,1,0.021297,0,0,0,-1,-21.591090,0,1
337352,19515412693,200,4871,0,1,38,38,0,12278.447368,0.027027,...,0,1,0.017537,0,0,0,-1,-22.331652,0,1
337353,11122514888,301,302,1,1,17,17,0,19929.058824,1652.750000,...,0,0,0.021529,0,1,1,-1,-70.092258,0,1
337354,11122514888,200,11438,1,1,17,17,0,19929.058824,1652.750000,...,0,0,0.032691,0,1,0,-1,-57.563937,0,1


In [19]:
import pandas as pd
# Define weights
weights = {
    'kmeans_result': 0.30,
    'DBSCAN_Anomaly': 0.30,
    'Isolation Forest Anomaly Prediction': 0.1,
    'LOF_Prediction': 0.15,
    'anomaly': 0.15 ##GMM
}

# Calculate weighted score for unsupervised methods
logs_df['unsupervised_score'] = (logs_df['kmeans_result'] * weights['kmeans_result'] +
                            logs_df['DBSCAN_Anomaly'] * weights['DBSCAN_Anomaly'] +
                            logs_df['Isolation Forest Anomaly Prediction'] * weights['Isolation Forest Anomaly Prediction'] +
                            logs_df['LOF_Prediction'] * weights['LOF_Prediction']+
                            logs_df['anomaly'] * weights['anomaly'])

# Generate blacklist warning
logs_df['blacklist_warning'] = logs_df['is_blacklist'].apply(lambda x: 'Warning' if x == 1 else 'No Warning')



In [20]:
# Normalize relevant features for composite score
from sklearn.preprocessing import MinMaxScaler

features_to_normalize = ['n_nans', 'is_bot', 'n_requests', 'n_errors', 'anomaly_score']
scaler = MinMaxScaler()

logs_df[features_to_normalize] = scaler.fit_transform(logs_df[features_to_normalize])

# Create additional composite score
logs_df['composite_score'] = (logs_df['n_nans'] + logs_df['is_bot'] + logs_df['n_requests'] +
                         logs_df['n_errors'] + logs_df['anomaly_score']) / len(features_to_normalize)

logs_df[['unsupervised_score', 'composite_score']] = scaler.fit_transform(logs_df[['unsupervised_score', 'composite_score']])

# Final score
logs_df['final_score'] = (logs_df['unsupervised_score'] + logs_df['composite_score']) / 2

#data_sorted = data.sort_values(by='final_score', ascending=False)
# Display the DataFrame with the final score and warnings
final_data = logs_df[['ip','final_score', 'blacklist_warning']]
final_data


,ip,final_score,blacklist_warning
0,5488240137,0.345564,No Warning
1,5488240137,0.347853,No Warning
2,11024920167,0.350178,No Warning
3,11024920167,0.296333,No Warning
4,11024920167,0.296869,No Warning
...,...,...,...
337351,19515412693,0.148171,Warning
337352,19515412693,0.147629,Warning
337353,11122514888,0.395469,No Warning
337354,11122514888,0.347080,No Warning


In [5]:
import zipfile
import os

# Path to the zip file
zip_path = 'LSTM_SYNTESIS_SCORES.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('extracted_SCORE_LSTM')

In [6]:
# Load the new file containing IP, LSTM score, LSTM loss, and log lines
new_file_path = 'extracted_SCORE_LSTM/LSTM_SYNTESIS_SCORES/sequence_anomaly_scores.csv'
new_data_df = pd.read_csv(new_file_path)
new_data_df

,Unnamed: 0,Log lines,ip,Loss_mse,Anomaly score
0,0,"[26357, 26403, 40459, 40464, 45616, 62926, 692...",38142,0.299414,0.000000
1,1,"[26403, 40459, 40464, 45616, 62926, 69260, 693...",38142,0.299914,0.000000
2,2,"[40459, 40464, 45616, 62926, 69260, 69306, 693...",38142,0.298054,0.000000
3,3,"[40464, 45616, 62926, 69260, 69306, 69332, 967...",38142,0.298197,0.000000
4,4,"[45616, 62926, 69260, 69306, 69332, 96775, 967...",38142,0.280276,0.000000
...,...,...,...,...,...
300276,300276,"[195509, 195510, 195511, 195512, 209980, 20998...",217128213115,0.107199,0.000000
300277,300277,"[195510, 195511, 195512, 209980, 209981, 20998...",217128213115,0.107488,0.000000
300278,300278,"[248193, 248195, None, None, None, None, None,...",220135220239,0.209106,0.000000
300279,300279,"[78805, 146702, None, None, None, None, None, ...",220181108176,5.247512,0.987223


In [15]:
import ast

In [24]:
# Add new columns to the logs DataFrame
logs_df['LSTM_Score'] = 0.0
logs_df['LSTM_Loss'] = 0.0
logs_df['Count'] = 0  # To keep track of how many times each row is updated

# Iterate over each row in the new data
for index, row in new_data_df.iterrows():
    ip = row['ip']
    score = row['Anomaly score']
    loss = row['Loss_mse']
    log_lines_str = row['Log lines']  # Get the string representation of log lines
    
    # Safely evaluate the string representation to a list of integers
    log_lines_list = ast.literal_eval(log_lines_str)
    print(log_lines_list)
    # Update the logs DataFrame
    for log_line in log_lines_list:
        mask = (final_data.index == log_line) & (logs_df['ip'] == ip)  # Corrected column names
        logs_df.loc[mask, 'LSTM_Score'] += score
        logs_df.loc[mask, 'LSTM_Loss'] += loss
        logs_df.loc[mask, 'Count'] += 1

# Calculate the mean score and loss
logs_df['LSTM_Score'] = logs_df.apply(lambda row: row['LSTM_Score'] / row['Count'] if row['Count'] > 0 else 0, axis=1)
logs_df['LSTM_Loss'] = logs_df.apply(lambda row: row['LSTM_Loss'] / row['Count'] if row['Count'] > 0 else 0, axis=1)

# Drop the Count column as it's no longer needed
logs_df = logs_df.drop(columns=['Count'])

# Save the updated DataFrame to a new CSV file
updated_log_file_path = 'final_scores.csv'
logs_df.to_csv(updated_log_file_path, index=False)


[26357, 26403, 40459, 40464, 45616, 62926, 69260, 69306, 69332, 96775]


KeyError: 0